Here, we'll have a look at [WildChat Dataset](https://wildchat.allen.ai/). 

[Huggingface dataset](https://huggingface.co/datasets/allenai/WildChat-1M) (hugging face stub: `allenai/WildChat-1M`)


In [4]:
from imbed_data_prep.wildchat import *



# Appendix

## Making the raw data

In [55]:
from dataclasses import dataclass, KW_ONLY
from imbed.base import HugfaceDaccBase

d = HugfaceDaccBase("allenai/WildChat-1M", name='wildchat')



In [58]:
df = d.train_data

In [59]:
df.shape

(840774, 14)

In [60]:
df.iloc[0]

conversation_hash                       c9ec5b440fbdd2a269333dd241f32f64
model                                                         gpt-4-0314
timestamp                                      2023-04-09 00:02:53+00:00
conversation           [{'content': 'Hey there! Are you familiar with...
turn                                                                   1
language                                                         English
openai_moderation      [{'categories': {'harassment': False, 'harassm...
detoxify_moderation    [{'identity_attack': 0.00020589135237969458, '...
toxic                                                              False
redacted                                                           False
state                                                              Texas
country                                                    United States
hashed_ip              22fd87ba9b98f3d379b23c7b52961f2d4a8505127e58b3...
header                 {'accept-language': 'en-US,e

In [61]:
from collections import Counter
Counter(df.model)

Counter({'gpt-3.5-turbo-0613': 371341,
         'gpt-3.5-turbo-0301': 195161,
         'gpt-4-1106-preview': 100972,
         'gpt-4-0125-preview': 62581,
         'gpt-3.5-turbo-0125': 57426,
         'gpt-4-0314': 53291,
         'gpt-4-0613': 2})

In [65]:
t = df.conversation.iloc[0]
t

array([{'content': 'Hey there! Are you familiar with reality shifting? So, I’m refining a foolproof method for reality shifting and want to pick a destination. Want to help me? I’m thinking something pretty personalized. There are a few things that are required of my destination. 1. The quest. I have to have a clear overarching goal in my reality, and don’t make it too crazy. It should be more along the lines of “save the president’s daughter” or “escape this weird wacky sinister place” NOT “get an artifact that literally controls reality”. Seriously, don’t make me fetch an artifact, or fetch anything. Instead, make me DO something. 2. Babes. I need pretty girls. 3. The entry. I need to get to lose consciousness in order to begin my journey in my desired reality, preferably by having it knocked out by one of the aforementioned babes. 4. Action. It needs to be cool. 5. Unconsciousness. Myself and the babes need to pass out in this place, preferably by being knocked out in some way or fa

In [66]:
t[0]

{'content': 'Hey there! Are you familiar with reality shifting? So, I’m refining a foolproof method for reality shifting and want to pick a destination. Want to help me? I’m thinking something pretty personalized. There are a few things that are required of my destination. 1. The quest. I have to have a clear overarching goal in my reality, and don’t make it too crazy. It should be more along the lines of “save the president’s daughter” or “escape this weird wacky sinister place” NOT “get an artifact that literally controls reality”. Seriously, don’t make me fetch an artifact, or fetch anything. Instead, make me DO something. 2. Babes. I need pretty girls. 3. The entry. I need to get to lose consciousness in order to begin my journey in my desired reality, preferably by having it knocked out by one of the aforementioned babes. 4. Action. It needs to be cool. 5. Unconsciousness. Myself and the babes need to pass out in this place, preferably by being knocked out in some way or fainting.

In [67]:
from tabled import expand_rows

w = expand_rows(df.iloc[:10], ['conversation', 'openai_moderation', 'detoxify_moderation'])
w.shape

(24, 14)

In [68]:
w.conversation.iloc[0]

{'content': 'Hey there! Are you familiar with reality shifting? So, I’m refining a foolproof method for reality shifting and want to pick a destination. Want to help me? I’m thinking something pretty personalized. There are a few things that are required of my destination. 1. The quest. I have to have a clear overarching goal in my reality, and don’t make it too crazy. It should be more along the lines of “save the president’s daughter” or “escape this weird wacky sinister place” NOT “get an artifact that literally controls reality”. Seriously, don’t make me fetch an artifact, or fetch anything. Instead, make me DO something. 2. Babes. I need pretty girls. 3. The entry. I need to get to lose consciousness in order to begin my journey in my desired reality, preferably by having it knocked out by one of the aforementioned babes. 4. Action. It needs to be cool. 5. Unconsciousness. Myself and the babes need to pass out in this place, preferably by being knocked out in some way or fainting.

In [69]:
w.openai_moderation.iloc[0]

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'harassment_threatening': False,
  'hate': False,
  'hate/threatening': False,
  'hate_threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'self_harm': False,
  'self_harm_instructions': False,
  'self_harm_intent': False,
  'sexual': False,
  'sexual/minors': False,
  'sexual_minors': False,
  'violence': False,
  'violence/graphic': False,
  'violence_graphic': False},
 'category_scores': {'harassment': 0.000484861753648147,
  'harassment/threatening': 0.00012471186346374452,
  'harassment_threatening': 0.00012471186346374452,
  'hate': 0.000457498652394861,
  'hate/threatening': 1.3141398994775955e-05,
  'hate_threatening': 1.3141398994775955e-05,
  'self-harm': 5.52945930394344e-05,
  'self-harm/instructions': 1.1666821819744655e-06,
  'self-harm/intent': 1.4811689652560744e-05,
  'self_harm': 5.52945930394344e-05,
  'self_harm_instructions': 1.16668218

In [70]:
w.detoxify_moderation.iloc[0]

{'identity_attack': 0.00020589135237969458,
 'insult': 0.002148400293663144,
 'obscene': 0.0004123652761336416,
 'severe_toxicity': 3.0470857382169925e-05,
 'sexual_explicit': 0.00012746300490107387,
 'threat': 6.507332727778703e-05,
 'toxicity': 0.005422386806458235}

In [71]:
from tabled import expand_columns

ww = expand_columns(w, ['conversation', 'openai_moderation', 'detoxify_moderation'])
www = expand_columns(ww, ['openai_moderation.categories', 'openai_moderation.category_scores'])
ww.shape, www.shape

((24, 32), (24, 66))

In [72]:
# remove all columns that have a / in them (because they have duplicates)
www = www[[c for c in www.columns if '/' not in c]]
www.shape

(24, 54)

In [75]:
from tabled import expand_rows, expand_columns

def flatten_wildchat_data(df):
    # for each (aligned) item of conversation and moderation, make a new row
    df = expand_rows(df, ['conversation', 'openai_moderation', 'detoxify_moderation'])
    # These items are dicts, so expand their fields into columns
    df = expand_columns(df, ['conversation', 'openai_moderation', 'detoxify_moderation'])
    # # openai_moderation has yet one more level of nesting so flatten that too
    # df = expand_columns(df, ['openai_moderation.categories', 'openai_moderation.category_scores'])
    # # remove all columns that have a / in them (because they have duplicates)
    # df = df[[c for c in df.columns if '/' not in c]]
    return df

In [76]:
df = flatten_wildchat_data(df)

In [77]:
df.to_parquet('wildchat_train.parquet')

In [6]:
data_for_cosmo = d.data_with_planar_embeddings()

Making dataframe from parquet bytes
Loading bytes of raw data from /Users/thorwhalen/Dropbox/_odata/figiri/github-repos.parquet
... Done
Dropping duplicate nameWithOwner (github stub)...
... Dropped 209524 duplicates
... Done


In [6]:
# d.cache['github_repo_for_cosmos.parquet'] = data_for_cosmo

In [19]:
# get numpy array of segment vectors
# the d.segment_vectors is a series whose elements are numpy arrays
# We want to have a 2 dimentional array with the same number of rows as the series
# and the number of columns as the length of the numpy arrays

import numpy as np

X = np.vstack(d.segment_vectors.tolist())
X.shape

(3065063, 256)

In [15]:
from sklearn.cluster import KMeans
d.embeddings_matrix

(3065063, 256)

In [9]:
t = d.cluster_indices

Computing cluster indices for num of clusters: (5, 8, 13, 21, 34)
... Done


# Data prep misc

In [ ]:
from imbed_data_prep.github_repos import *

d = GithubReposData()

## Can we take the github stub (nameWithOwner) as an id?

In [18]:
assert len(d.raw_data.nameWithOwner.dropna()) == len(d.raw_data), (
    "We assumed that there's no missing data in the 'nameWithOwner' column"
)

Making dataframe from parquet bytes
Loading bytes of raw data from /Users/thorwhalen/Dropbox/_odata/figiri/github-repos.parquet


In [24]:
assert d.raw_data.nameWithOwner.nunique() == len(d.raw_data), (
    "We assumed that the 'nameWithOwner' were unique"
)

AssertionError: We assumed that the 'nameWithOwner' were unique

### Let's look at the duplicates

In [25]:
from collections import Counter

Counter(d.raw_data.nameWithOwner).most_common(10)

[('plapadoo/ledger-jira-sync', 4),
 ('wasp-platform/wasp', 4),
 ('dimdenGD/OldTweetDeck', 3),
 ('TravisWThompson1/Makefile_Example_CUDA_CPP_To_Executable', 3),
 ('Roave/EmailTemplates', 3),
 ('grigio/meteor-weuse', 3),
 ('laewahn/CutCAD', 3),
 ('cockroachdb/cdc-sink', 3),
 ('AP-Atul/Torpido', 3),
 ('althonos/pyhmmer', 3)]

In [26]:
d.raw_data[d.raw_data.nameWithOwner == "plapadoo/ledger-jira-sync"]

,owner,name,stars,forks,watchers,isFork,isArchived,languages,languageCount,topics,...,createdAt,pushedAt,defaultBranchCommitCount,license,assignableUserCount,codeOfConduct,forkingAllowed,nameWithOwner,parent,embedding
2117813,plapadoo,ledger-jira-sync,6,0,6,False,False,"Python: 8602, Nix: 1530",2,"ledger: 15, python: 277441, accounting: 67, bo...",...,2018-05-19T08:00:12Z,2019-03-05T17:55:25Z,9.0,"BSD 3-Clause ""New"" or ""Revised"" License",3,None,True,plapadoo/ledger-jira-sync,NaN,"[-0.04487917, 0.025739525, 0.106529795, -0.135..."
2117814,plapadoo,ledger-jira-sync,6,0,6,False,False,"Python: 8602, Nix: 1530",2,"ledger: 15, python: 277441, accounting: 67, bo...",...,2018-05-19T08:00:12Z,2019-03-05T17:55:25Z,9.0,"BSD 3-Clause ""New"" or ""Revised"" License",3,None,True,plapadoo/ledger-jira-sync,NaN,"[-0.04487917, 0.025739525, 0.106529795, -0.135..."
2117815,plapadoo,ledger-jira-sync,6,0,6,False,False,"Python: 8602, Nix: 1530",2,"ledger: 15, python: 277441, accounting: 67, bo...",...,2018-05-19T08:00:12Z,2019-03-05T17:55:25Z,9.0,"BSD 3-Clause ""New"" or ""Revised"" License",3,None,True,plapadoo/ledger-jira-sync,NaN,"[-0.04487917, 0.025739525, 0.106529795, -0.135..."
2117816,plapadoo,ledger-jira-sync,5,0,6,False,False,"Python: 8602, Nix: 1530",2,"ledger: 15, python: 277500, accounting: 67, bo...",...,2018-05-19T08:00:12Z,2019-03-05T17:55:25Z,9.0,"BSD 3-Clause ""New"" or ""Revised"" License",3,None,True,plapadoo/ledger-jira-sync,NaN,"[-0.04487917, 0.025739525, 0.106529795, -0.135..."


It looks like the meta data is exactly the same, so the non-unique entries might be duplicates. Let's drop any rows that are exact duplicates (except for the embeddings, which are numpy arrays, so not hashable, and could have slight variations). 

In [35]:
# drop duplicates, ignoring the embedding column (which is not hashable)
t = d.raw_data.drop_duplicates(subset=d.raw_data.columns.difference(["embedding"]))
assert d.raw_data.shape[1] == t.shape[1], "Assumed no columns were dropped"
print(f"{len(d.raw_data) - len(t)} rows were dropped")
d.raw_data.shape, t.shape

207147 rows were dropped


((3274587, 26), (3067440, 26))

In [36]:
# Still some duplicate nameWithOwner, with different stars etc.
# So we'll just drop duplicate nameWithOwner
assert t.nameWithOwner.nunique() == len(t), (
    "We assumed that the 'nameWithOwner' were unique"
)

AssertionError: We assumed that the 'nameWithOwner' were unique

In [39]:
Counter(t.nameWithOwner).most_common(10)

[('dimdenGD/OldTweetDeck', 3),
 ('weaigc/gradio-chatbot', 3),
 ('MoyuruAizawa/Cropify', 3),
 ('Lakr233/BBackupp', 3),
 ('daboigbae/react-native-template', 3),
 ('esp-rs/esp-mbedtls', 3),
 ('Nahid1911/space-travelers-proj-react', 3),
 ('Rel1cx/eslint-plugin-react-ts', 3),
 ('redknotmiaoyuqiao/EyerH264Decoder', 2),
 ('ShuifaHe/STM32', 2)]

In [40]:
d.raw_data[d.raw_data.nameWithOwner == 'dimdenGD/OldTweetDeck']

,owner,name,stars,forks,watchers,isFork,isArchived,languages,languageCount,topics,...,createdAt,pushedAt,defaultBranchCommitCount,license,assignableUserCount,codeOfConduct,forkingAllowed,nameWithOwner,parent,embedding
8747,dimdenGD,OldTweetDeck,129,5,10,False,False,JavaScript: 5093,1,,...,2023-08-17T00:31:05Z,2023-08-18T19:55:14Z,14.0,MIT License,1,None,True,dimdenGD/OldTweetDeck,NaN,"[-0.093423545, -0.07952896, -0.038683187, -0.0..."
8748,dimdenGD,OldTweetDeck,97,4,8,False,False,JavaScript: 5093,1,,...,2023-08-17T00:31:05Z,2023-08-18T16:44:39Z,13.0,MIT License,1,None,True,dimdenGD/OldTweetDeck,NaN,"[-0.093423545, -0.07952896, -0.038683187, -0.0..."
8749,dimdenGD,OldTweetDeck,128,5,10,False,False,JavaScript: 5093,1,,...,2023-08-17T00:31:05Z,2023-08-18T19:55:14Z,14.0,MIT License,1,None,True,dimdenGD/OldTweetDeck,NaN,"[-0.093423545, -0.07952896, -0.038683187, -0.0..."


## Different ways to get an array out of a dataframe

In [3]:
import pandas as pd
import numpy as np
t = pd.DataFrame(np.random.randint(0, 10000, (10000, 2)), columns=['a', 'b'])

In [12]:
%timeit tt = t.a.to_numpy()  # 3.38 µs ± 10.1 ns

3.38 µs ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
%timeit tt = np.array(t.a.to_list())  # 351 µs ± 2.92 µs

351 µs ± 2.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# Scrap: Github Repos Embeddings

In [1]:
from imbed_data_prep.github_repos import _DFLT_RAW_DATA_FILEPATH, _DFLT_CACHE_DIR
from imbed_data_prep.github_repos import *

In [1]:
log_flag_attr = 'verbose'
args = ()
getattr(next(iter(args), None), log_flag_attr, False)

False

In [4]:
from lkj import log_calls

class MyClass:
    def __init__(self, verbose=False):
        self.verbose = verbose

    @log_calls(log_flag_attr='verbose')
    def foo(self):
        print("Executing foo")

# Example usage
obj = MyClass(verbose=True)
obj.foo()  # This will log


Calling foo...
Executing foo
.... Done calling foo
